In [ ]:
from pymatgen.ext.matproj import MPResterimport pandas as pdAPI_KEY = 'YOUR_API_KEY'with MPRester(API_KEY) as mpr:    # Define the elements you want to query    elements = ['Ga', 'Tm', 'Y', 'Dy', 'Nb', 'Pu']        all_materials = []        for element in elements:        # Construct the query for the element        query = {"elements": {$all: [element]}}                # Get the summary documents for the query        materials = mpr.summary.search(query)                print(f"Querying elements: {', '.join(elements)}")        print(f"  - Querying for materials with {element}")        print(f"    Found {len(materials)} materials")        all_materials.extend(materials)        print(f"\nTotal materials found: {len(all_materials)}")        # Convert the list of materials to a Pandas DataFrame    df = pd.DataFrame([material.dict() for material in all_materials])        # Print the DataFrame    print(df.head())        # Save the DataFrame to a file    # df.to_json('magnetic_materials.json')        # You can now work with the 'df' DataFrame to further analyze the data    # or perform other operations as needed        # Example: Display the number of materials found for each element    # element_counts = df['elements'].apply(lambda x: x[0]).value_counts()    # print(element_counts)        # Replace 'your_property' with the actual property you want to track    # property_counts = df['your_property'].value_counts()    # print(property_counts)        # Replace 'your_property' with the actual property you want to track    # property_values = df['your_property'].unique()    # print(property_values)        # Replace 'your_property' with the actual property you want to track    # property_descs = df['your_property'].describe()    # print(property_descs)        # Replace 'your_property' with the actual property you want to track    # property_info = df['your_property'].info()    # print(property_info)        # Replace 'your_property' with the actual property you want to track    # property_nulls = df['your_property'].isnull().sum()    # print(property_nulls)        # Replace 'your_property' with the actual property you want to track    # property_duplicates = df['your_property'].duplicated().sum()    # print(property_duplicates)        # Replace 'your_property' with the actual property you want to track    # property_unique_values = df['your_property'].unique()    # print(property_unique_values)        # Replace 'your_property' with the actual property you want to track    # property_value_counts = df['your_property'].value_counts()    # print(property_value_counts)        # Replace 'your_property' with the actual property you want to track    # property_missing_values = df['your_property'].isnull().sum()    # print(property_missing_values)        # Replace 'your_property' with the actual property you want to track    # property_data_types = df['your_property'].dtype    # print(property_data_types)        # Replace 'your_property' with the actual property you want to track    # property_summary_stats = df['your_property'].describe()    # print(property_summary_stats)        # Replace 'your_property' with the actual property you want to track    # property_correlation = df['your_property'].corr()    # print(property_correlation)        # Replace 'your_property' with the actual property you want to track    # property_distribution = df['your_property'].hist()    # print(property_distribution)        # Replace 'your_property' with the actual property you want to track    # property_box_plot = df['your_property'].plot(kind='box')    # print(property_box_plot)        # Replace 'your_property' with the actual property you want to track    # property_scatter_plot = df.plot(x='element', y='your_property', kind='scatter')    # print(property_scatter_plot)        # Replace 'your_property' with the actual property you want to track    # property_time_series_plot = df.plot(x='date', y='your_property')    # print(property_time_series_plot)